**Hidden Markov models for cracking codes**

In this exercise you have to make a partially built HMM work and use it to solve some simple substitution ciphers. Plaintext data is provided in 'plaintext' directory. Encrypted data is in 'encrypted'. Some of the texts were originally English some of them were Russian; the sequences are also of different lengths. 

This homework is worth **15 points** and is due by the next class (**24th Oct.**), please submit the results of the **TASK 5** (a list of files and names of the author/work) to Anytask in the following format: 'filename author' where 'filename' is a file from "encrypted/\*_encrypted.txt" and 'author' is a file from "plaintext/\*.txt" (not including 'english.txt', 'russian.txt' or 'all.txt') which best matches the decrypted text.




In [22]:
# Utilities for loading data from file and converting characters to integers and back.
import numpy as np
    
def get_char_to_int_mapping(path):
    # Load data from path and get mapping from characters to integers and back.
    characters = set()
    for line in open(path):
        characters.update(set([c for c in line.strip()]))
    char_to_int_mapping = dict([(char, i) for i, char in enumerate(sorted(list(characters)))])
    int_to_char_mapping = [char for char, i in char_to_int_mapping.items()]
    return char_to_int_mapping, int_to_char_mapping

def load_sequences(path, char_to_int_mapping):
    # Load data from path and map to integers using mapping.
    return [[char_to_int_mapping[c] for c in line.strip()] for line in open(path)]

def estimate_markov_model_from_sequences(sequences, num_states):
    # Estimate a Markov model based on the sequences (integers) provided.

    # pi[i] = Pr(s_0 = i)
    pi_counts = np.zeros(num_states)

    # A[i, j] = Pr(s_t = j | s_{t-1} = i)
    A_counts = np.zeros((num_states, num_states))
    move_out_count = np.zeros(num_states)

    for n, sequence in enumerate(sequences):
        for prev, nxt in zip(sequence[:-1], sequence[1:]):
            A_counts[prev, nxt] += 1
            pi_counts[prev] += 1
        if len(sequence) > 0:
            pi_counts[sequence[-1]] += 1
            
    pi = pi_counts / np.sum(pi_counts)
    A = A_counts / np.sum(A_counts, axis=1)[:, None]
    
    return pi, A

**TASK 1**: Make the following block run by completing the method 'estimate_markov_model_from_sequences' above.

In [14]:
%%time
# Some data to use.
plaintext = 'plaintext/english.txt'
# plaintext = 'plaintext/shakespeare.txt'
# plaintext = 'plaintext/russian.txt'

ciphertext = 'encrypted/1_encrypted.txt' # short sequences in english
# ciphertext = 'encrypted/99_encrypted.txt' # longer sequences in russian

# load a character to integer mapping and reverse                                                                                                         
char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext)

# load sequences as ints                                                                                                                                  
plaintext_sequences = load_sequences(plaintext, char_to_int_mapping)
encrypted_sequences = load_sequences(ciphertext, char_to_int_mapping)
# estimate a markov model over characters                                                                                                                 
pi, A = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))


CPU times: user 5.68 s, sys: 24 ms, total: 5.71 s
Wall time: 5.7 s


Below is a mostly implemented HMM.

In [15]:
from IPython.display import clear_output

In [27]:
class HMM():

    def __init__(self, observations_to_char_mapping={}, states_to_char_mapping={}, max_len=60):
        # Determine number of states and observation space. 
        self.num_states = len(states_to_char_mapping) # number of states = number of chars 
        self.num_outputs = len(observations_to_char_mapping) # number of outputs ?
        self.states_to_char_mapping = states_to_char_mapping # get char from state
        self.observations_to_char_mapping = observations_to_char_mapping # what is observations ??? 
        
        self.max_len = max_len ## MAX LEN OF ENCRIPTED TEXT
        # Random initialization
        self.pi = np.random.rand(self.num_states)# init pi with random 
        self.pi /= np.sum(self.pi)
        self.A = np.random.rand(self.num_states, self.num_states).astype('float128') # init transition matrix with random 
        self.A /= np.sum(self.A, 1, keepdims=True)
        self.B = np.random.rand(self.num_states, self.num_outputs).astype('float128') # init emission matrix 
        self.B /= np.sum(self.B, 1, keepdims=True)
        
        
    def estimate_with_em(self, sequences, parameters={}, epsilon=0.001, max_iters=100):
        # Estimates all parameters not provided in 'parameters' based on 'sequences'.
        self.fixed_pi = 'pi' in parameters
        self.pi = parameters['pi'] if self.fixed_pi else self.pi 
        
        self.fixed_A = 'A' in parameters
        self.A = parameters['A'] if self.fixed_A else self.A
        
        self.fixed_B = 'B' in parameters
        self.B = parameters['B'] if self.fixed_B else self.B
           
        previous_llh = None # prev log-likelihood
        
        iteration = 0
        while True and iteration < max_iters: # why we dont use for-loop here? 
            # Infer expected counts.
            pi_counts, A_counts, B_counts, log_likelihood = self.e_step(sequences) # E-step from EM-algo

            # Update parameters based on counts.
            self.m_step(pi_counts, A_counts, B_counts) # M-step from EM-algo

            # Output some sequences for debugging.
            if iteration % 10 == 0:
                # clear_output()
                print('iteration %d; log likelihood %.4f' % (iteration, log_likelihood))
                # self.output(sequences[:10])

            # Log likelihood should be increasing
            if previous_llh:
                assert log_likelihood >= previous_llh
                if log_likelihood - previous_llh < epsilon:
                    break
            previous_llh = log_likelihood
        
            iteration += 1
        return previous_llh, self.output(sequences[:10])

    def e_step(self, sequences):
        # Reset counters of statistics
        pi_counts = np.zeros_like(self.pi)
        A_counts = np.zeros_like(self.A) 
        B_counts = np.zeros_like(self.B) 
        total_log_likelihood = 0.0

        for sequence in sequences:
            for i in range(0, len(sequence), self.max_len):
                sub_seq = sequence[i:i+self.max_len] 

                # Run Forward-Backward dynamic program
                alpha, beta, gamma, xi, log_likelihood = self.forward_backward(sub_seq)

                # Accumulate statistics.
                pi_counts += gamma[:, 0]
                A_counts += xi
                for t, x in enumerate(sub_seq):
                    B_counts[:, x] += gamma[:, t]

                total_log_likelihood += log_likelihood

        return pi_counts, A_counts, B_counts, total_log_likelihood

    def m_step(self, pi_counts, A_counts, B_counts):
        if not self.fixed_pi:
            self.pi = pi_counts / np.sum(pi_counts)
        if not self.fixed_A:
            self.A = A_counts / np.sum(A_counts, 1, keepdims=True)
        if not self.fixed_B:
            self.B = B_counts / np.sum(B_counts, 1, keepdims=True)
        
    def max_posterior_decode(self, sequence):
        _, _, gamma, _, log_likelihood = self.forward_backward(sequence)
        return np.argmax(gamma, 0)
        
    def forward_backward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha = self.forward(sequence) 
        
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = self.backward(sequence)

        # gamma[i][t] = p(z_t = i|x_1, ..., x_T)
        ss = np.sum(alpha * beta, 0, dtype=np.float128)
        if (ss == 0).any():
            print('err')
        assert np.isfinite(ss).all()
        gamma = (alpha * beta) / ss

        # xi[i][j] = p(z_t = i, z_{t+1} = j|x_1, ..., x_T)
        xi = np.zeros_like(self.A)
        for t in range(1, len(sequence)-1):
            this_xi = np.zeros_like(self.A) ## Changed for better performance 
            this_xi += (self.A * alpha[:, t][:, None]) * (beta[:, t+1] * self.B[:, sequence[t+1]])      
            xi += this_xi / np.sum(this_xi)
                
        return alpha, beta, gamma, xi, np.log(np.sum(alpha[:, len(sequence)-1]))

    def forward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha = np.zeros((len(self.pi), len(sequence)), dtype=np.float128)
        alpha[:, 0] = self.pi * self.B[:, sequence[0]]
        for t in range(len(sequence) - 1):
            x = sequence[t+1]
            alpha[:, t+1] = self.B[:, x] * np.sum(self.A * alpha[:, t][:, None], axis=0, dtype=np.float128)
        return alpha 
    
    def backward(self, sequence):
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = np.zeros((len(self.pi), len(sequence)))
        beta[:, -1] = 1
        
        for i, x in enumerate(sequence[1:][::-1]):
            t = len(sequence) - 1 - i
            beta[:, t-1] = np.sum(self.A * (beta[:, t] * self.B[:, x]), axis=1, dtype=np.float128)
        
        return beta

    def output(self, sequences):
        # Output some decoded states. 
        res = []
        for i, sequence in enumerate(sequences):
            observations = []
            map_states = []
            for i in range(0, len(sequence), self.max_len):
                sub_seq = sequence[i:i+self.max_len] 
                sub_observations = [self.observations_to_char_mapping[x] for x in sub_seq]                
                sub_map_states = [self.states_to_char_mapping[x] for x in self.max_posterior_decode(sub_seq)]
                observations.append(''.join(sub_observations))
                map_states.append(''.join(sub_map_states))
                
            print('(states):       %s\n(observations): %s' % (''.join(map_states), ''.join(observations)))
            res.append('(states):       %s\n(observations): %s' % (''.join(map_states), ''.join(observations)))
        return res


**TASK 2**: Implement the assertions in 'forward' and 'backward' methods on the HMM class so that the following block passes.

In [17]:
# Since it's a substitution cipher we assume hidden states and observations have same alphabet.
state_to_char_mapping = int_to_char_mapping
observation_to_char_mapping = int_to_char_mapping
# Initialize a HMM with the correct state/output spaces.
hmm = HMM(observation_to_char_mapping, state_to_char_mapping)

# Estimate the parameters and decode the encrypted sequences.
llh, output = hmm.estimate_with_em(encrypted_sequences[:100], parameters={'pi': pi, 'A': A})

iteration 0; log likelihood -12078.0470
iteration 10; log likelihood -9866.1163
iteration 20; log likelihood -9360.5749
iteration 30; log likelihood -8951.6571
iteration 40; log likelihood -8733.8902
iteration 50; log likelihood -8687.9314
iteration 60; log likelihood -8680.0509
iteration 70; log likelihood -8675.7187
iteration 80; log likelihood -8670.7223
iteration 90; log likelihood -8666.3393
(states):       than such a roman
(observations): noeixjtcoxexhwfei
(states):       cassous brutus bait not me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill not endure it you fonget yourself
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a soldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in prackice abler than yourself
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to mave conditions
(observations): nwxfeaqxcwi gngwij
(states):       brutus go to you are not cassius
(observations):

### Task2 results 
We can see that hmm works good enough with small english texts

p.s. this is work of algorithm with task3-4 improvements

**TASK 3**: Some of the encrypted sequences are quite long. Try decoding some from 'encrypted/99_encrypted.txt' (note these are in Russian).

In [28]:
plaintext_russian = 'plaintext/russian.txt'
ciphertext_russian = 'encrypted/99_encrypted.txt'

char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext_russian)

plaintext_sequences = load_sequences(plaintext_russian, char_to_int_mapping)
encrypted_sequences = load_sequences(ciphertext_russian, char_to_int_mapping)
pi_ru, A_ru = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))



state_to_char_mapping = int_to_char_mapping
observation_to_char_mapping = int_to_char_mapping
hmm = HMM(observation_to_char_mapping, state_to_char_mapping, max_len=15)

llh, output = hmm.estimate_with_em(encrypted_sequences[:30], parameters={'pi': pi_ru, 'A': A_ru}, max_iters=50)
# take first 30 encripted examples to reduce time of processing
# for the same reason change the number of iterations

iteration 0; log likelihood -22996.2079
iteration 10; log likelihood -18070.3598
iteration 20; log likelihood -16542.6550
iteration 30; log likelihood -15735.2477
iteration 40; log likelihood -15635.3415
(states):       князь андрей пожал плечами и поморщился как морщатся любители музыки услышав пальшивую ноту обе женщины отпустили друг друга потом опять как бучто боясь опоздать схватили друг друга за руки стали целовать и отрывать руки и потом опять стали целовать друг друга в лицо и совершенно неожиданно для князя андрея обе заплакали и опять стали целоваться тоже 1921919ла князю андрею было очевидно неловко но для двух женщин казалось так естественно что они плакали казалось они и не предполагали чтобы могло иначе совершиться это свидание
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяр

## Task2 Results
- Hmm works quite good for that section, but we needed to change 

## Done

**TASK 4**: Make your implementation of forward and backward more efficient by removing all but the outermost for-loop.

## Done 

**TASK 5**: Try to classify the author of each text. 

In [352]:
def get_russian():
    russian_list = []
    for i in range(143):
        try:
            ciphertext_find = 'encrypted/{}_encrypted.txt'.format(i)
            encrypted_sequences_tt = load_sequences(ciphertext_find, char_to_int_mapping_tt)
            russian_list.append(i)
        except:
            continue
    return russian_list
    

In [523]:
rl = get_russian()
rl

[0,
 1,
 2,
 3,
 4,
 6,
 7,
 8,
 9,
 10,
 12,
 13,
 15,
 16,
 18,
 20,
 21,
 22,
 23,
 24,
 25,
 28,
 29,
 30,
 31,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 81,
 82,
 83,
 84,
 85,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 98,
 100,
 101,
 102,
 104,
 106,
 109,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 124,
 126,
 127,
 130,
 131,
 132,
 133,
 134,
 136,
 138,
 141,
 142]

In [522]:
plaintext_find = 'plaintext/english.txt'
char_to_int_mapping_tt, int_to_char_mapping_tt = get_char_to_int_mapping(plaintext_find)
plaintext_sequences_tt = load_sequences(plaintext_find, char_to_int_mapping_tt)
pi_tt, A_tt = estimate_markov_model_from_sequences(plaintext_sequences_tt, len(char_to_int_mapping_tt))

In [395]:

ciphertext_find = 'encrypted/26_encrypted.txt' # longer sequences in russian
encrypted_sequences_tt = load_sequences(ciphertext_find, char_to_int_mapping_tt)


In [527]:
MAX_LEN = 60

def find_min(files, cipher_file):
    scores = []
    for plain_file in files:
        print(plain_file)
        char_to_int_mapping_tt, int_to_char_mapping_tt = get_char_to_int_mapping(plain_file)
        plaintext_sequences_tt = load_sequences(plain_file, char_to_int_mapping_tt)
        
        ciphertext_find = cipher_file # longer sequences in russian
        encrypted_sequences_tt = load_sequences(ciphertext_find, char_to_int_mapping_tt)
        
        pi_tt, A_tt = estimate_markov_model_from_sequences(plaintext_sequences_tt, len(char_to_int_mapping_tt))
        # Since it's a substitution cipher we assume hidden states and observations have same alphabet.
        state_to_char = int_to_char_mapping_tt
        observation_to_char = int_to_char_mapping_tt

        # Initialize a HMM with the correct state/output spaces.
        hmm_tt = HMM(observation_to_char, state_to_char)

        # Estimate the parameters and decode the encrypted sequences.
        llh, res = hmm_tt.estimate_with_em(encrypted_sequences_tt[:100], parameters={'pi': pi_tt, 'A': A_tt}, max_iters=60)
        scores.append((llh, res))
        clear_output()
    return scores

In [528]:
files = [
    'plaintext/dickens.txt',
    'plaintext/shakespeare.txt'
]

rr = []
for num in rl:
    res = find_min(files, 'encrypted/{}_encrypted.txt'.format(num))
    
    if res[0][0] > res[1][0]:
        print('dick ', num)
        print(res[0][1])
        rr.append([('dick', num), (res[0][1])])
    else:
        print('shec ', num)
        print(res[1][1])
        rr.append([('shec', num), (res[1][1])])


shec  142
['(states):       the heart of brothers govern in our loves\n(observations): noqxoqehnxwpxkhwnoqhjxvwzqhixgixwthxdwzqj', '(states):       and sway our great designs\n(observations): ei xjmebxwthxvhqenx qjgvij', '(states):       ckesar there is my han\n(observations): ceqjehxnoqhqxgjxfbxoei', '(states):       a sister i bequeath you whom no brother\n(observations): exjgjnqhxgxkqrtqenoxbwtxmowfxiwxkhwnoqh', '(states):       id iver love so dearly let her live\n(observations): g xqzqhxdwzqxjwx qehdbxdqnxoqhxdgzq', '(states):       to join our kingdomy and our hearts and never\n(observations): nwxlwgixwthxagiv wfjxei xwthxoqehnjxei xiqzqh', '(states):       fly off our loves again\n(observations): pdbxwppxwthxdwzqjxevegi', '(states):       lexidus happily amen\n(observations): dqyg tjxoeyygdbxefqi', '(states):       antond i did not thind to draw my sword gainst pompey\n(observations): einwibxgx g xiwnxnogiaxnwx hemxfbxjmwh xvegijnxywfyqb', '(states):       for he hath laid sthan

In [529]:
import pickle

In [530]:
with open('results', 'wb') as f:
    pickle.dump(rr, f)

In [545]:
data = []
for rs in rr:
#     print(rs[0])
    if rs[0][0] == 'shec':
        data.append('encrypted/{}_encrypted.txt '.format(rs[0][1]) +  'plaintext/shakespeare.txt\n')
    else:
        data.append('encrypted/{}_encrypted.txt '.format(rs[0][1]) +  'plaintext/dickens.txt\n')

In [546]:
with open('result_auth.txt', 'w', encoding='utf8') as f:
    f.writelines(data)